In [1]:
!pip install -q lightning flwr wandb hydra-core

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.0/819.0 kB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.0/540.0 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.4/242.4 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.0/236.0 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.2 MB/s e

In [4]:
from torch.utils.data import Dataset
from pathlib import Path
import nibabel as nib
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

In [5]:
from kaggle_secrets import UserSecretsClient
secret_label = "wandb-key"
WANDB_APIKEY = UserSecretsClient().get_secret(secret_label)

ROOT_PATH = '/kaggle/input/mri-dataset/datasetzip/not_skull_stripped'


In [12]:
label = pd.read_json('/kaggle/input/mri-dataset/mri_dataset.json')

# Dataset

In [21]:
import re 

class MRIDataset(Dataset) :

    def __init__(self, root_dir: str, label_path: str = None, transform = None, label_df: pd.DataFrame = None, is_3d: bool = False):
        self.root_dir = Path(root_dir)
        self.transform = transform
        self.is_3d = is_3d
        if label_df is None:
          self.labels_df = pd.read_csv(label_path)
          
        else :
          self.labels_df = label_df

        self.labels_df['subject_id'] = self.labels_df['subject_id'].astype(str)
        self.labels_df = self.labels_df[self.labels_df['subject_dx'] == 'control']

        
        fail_set = set([
            "sub-BrainAge005600_T1w.nii" 
        ])
        
        valid_subjects = set(self.labels_df['subject_id'].astype(str).values)
        subject_pattern = re.compile(r"sub-([A-Za-z0-9]+)")
    
        self.file_paths = sorted([
            fp for fp in self.root_dir.rglob("*.nii")
            if fp.is_file()
            and fp.name not in fail_set
            and (match := subject_pattern.search(str(fp))) 
            and match.group(1) in valid_subjects
        ])



    def __len__(self):
        return len(self.file_paths)


    def preprocessing_datapoint(self, img_data):

        mid_x = img_data.shape[0] // 2
        mid_y = img_data.shape[1] // 2
        mid_z = img_data.shape[2] // 2

        axial_slice = img_data[:, :, mid_z]
        coronal_slice = img_data[:, mid_y, :]
        sagittal_slice = img_data[mid_x, :, :]


        combined_data = np.stack([axial_slice, coronal_slice, sagittal_slice], axis=0)
        combined_data = torch.from_numpy(combined_data).float()

        if self.transform : combined_data = self.transform(combined_data)

        return combined_data




    def __getitem__(self, idx):
        img_path = self.file_paths[idx]
        file_path_str = str(img_path)

        subject_id = None
        valid_subjects_set = set(self.labels_df['subject_id'].values)


        for sid in valid_subjects_set:
            if sid in file_path_str:
                subject_id = sid
                break

        if subject_id is None:
            raise ValueError(f"Không tìm thấy subject_id cho file: {img_path}")

        metadata = self.labels_df.loc[self.labels_df['subject_id'] == subject_id].iloc[0].to_dict()

        img_data = nib.load(img_path).get_fdata()

        img_data = torch.from_numpy(img_data).float()

        label = 0
        if metadata['subject_sex'] == 'm' : label = 1

        if not self.is_3d:
            img_data = self.preprocessing_datapoint(img_data)

        return img_data,  label



def visualize_sample(dataset, idx):
    mri_data, label = dataset[idx]
    title = f"Label: {label}\n"
    plt.close('all')
    fig = plt.figure(figsize = (18, 6))

    if isinstance(mri_data, torch.Tensor):
        data = mri_data.squeeze().numpy()
    else:
        data = mri_data


    ax1 = fig.add_subplot(1, 3, 1)
    plt.imshow(data[0, :, :].T, cmap='gray', origin='lower')

    ax2 = fig.add_subplot(1, 3, 2)
    ax2.imshow(data[1, :, :].T, cmap='gray', origin='lower')

    ax3 = fig.add_subplot(1, 3, 3)
    ax3.imshow(data[2, :, :].T, cmap='gray', origin='lower')

    plt.suptitle(title)
    plt.tight_layout()
    plt.show()


In [23]:
dataset = MRIDataset(root_dir= '/kaggle/input/mri-dataset' , label_path = '/kaggle/input/mri-label/label.csv', is_3d = True)

In [6]:
dataset[0][0].shape 

torch.Size([130, 130, 130])

## Data splitting

In [7]:
from torch.utils.data import random_split 

def preprocessing_labels(df: pd.DataFrame, root_dir: str = ROOT_PATH):
    
    subject_list = []
    for root, dirs, files in os.walk(root_dir):
      for dir_name in dirs:
        if dir_name.startswith("sub-BrainAge"):
            subject_list.append(dir_name)


    return df[df['subject_id'].isin(subject_list)]


def prepare_data(data: pd.DataFrame):

  df = data.copy()
  df['age_group'] = pd.qcut(df['subject_age'], q = min(5, len(df)), labels = False)
  df['key'] = df.apply(lambda row : f"{row['age_group']}_{row['subject_sex']}", axis = 1)
  return df


def sampling_data(data, size, random_state ):

  samples = data.groupby('key', group_keys = False)


  samples = samples.apply(lambda x: x.sample(
      n = min(int(size / len(data['key'].unique())), len(x)),
      replace = len(x) < int(size / len(data['key'].unique())),
      random_state =  random_state
  ))


  if len(samples) < size:
    additional_samples = data.drop(samples.index).sample(
        n = min(size - len(samples), len(data) - len(samples)),
        replace = True,
        random_state = random_state
    )

    samples = pd.concat([samples, additional_samples])
  return samples


def create_train_test(sample_labels: list, val_ratio: float = 0.2, root_dir: str = ROOT_PATH, is_3d: bool = False):

  client_datasets = []
  for label_df in sample_labels:
    dataset = MRIDataset(root_dir=root_dir, label_df = label_df, is_3d = is_3d)
    
    train_dataset, val_dataset = random_split(dataset, [1 - val_ratio, val_ratio])
    client_datasets.append((train_dataset, val_dataset))
  return client_datasets


def distributed_data_to_clients(data: pd.DataFrame, num_clients: int, overlap_ratio: float):

  df = prepare_data(data)

  n_samples = len(df)
  samples_per_client = int(n_samples / (num_clients * (1 - overlap_ratio) + overlap_ratio))

  client_datasets = []
  selected_samples = {}

  # Tạo các client datasets với sự phân bố cân bằng
  for client_idx in range(num_clients):

      if client_idx == 0:
          client_data = df.sample(n=samples_per_client, random_state=42+client_idx)
      else:
          # overlap size
          overlap_size = int(samples_per_client * overlap_ratio)
          non_overlap_size = samples_per_client - overlap_size

          # building overlap
          all_previous_samples = pd.DataFrame()
          for prev_client_idx in range(client_idx):
              all_previous_samples = pd.concat([all_previous_samples, selected_samples[prev_client_idx]])

          # sampling
          if len(all_previous_samples) > 0:
              overlap_samples = sampling_data(all_previous_samples, overlap_size, client_idx * 100 + 42)
          else:
              overlap_samples = pd.DataFrame(columns=df.columns)

          # Lấy mẫu mới (không overlap)
          remaining_indices = df.index.difference(all_previous_samples.index)
          if len(remaining_indices) > 0:
              remaining_df = df.loc[remaining_indices]
              non_overlap_samples = sampling_data(remaining_df, non_overlap_size, client_idx * 100 + 42)
          else:

              non_overlap_samples = df.sample(n=non_overlap_size, replace=True, random_state=42+client_idx*300)


          client_data = pd.concat([overlap_samples, non_overlap_samples])


      selected_samples[client_idx] = client_data
      client_datasets.append(client_data.drop(['age_group', 'key'], axis=1))

  return client_datasets

In [8]:


def iid_client_split(dataset, num_client = 3,  val_ratio = 0.2):

    client_datasets = []
    sample_per_client = len(dataset) // num_client


    for i in range(num_client):
        start_idx = i * sample_per_client
        end_idx = (i + 1) * sample_per_client if i < num_client - 1 else len(dataset)
        indecies = list(range(start_idx, end_idx))

        client_dataset = torch.utils.data.Subset(dataset, indecies)
        train_dataset, val_dataset = random_split(client_dataset, [1 - val_ratio, val_ratio])

        client_datasets.append((train_dataset, val_dataset))
    return client_datasets





def same_distribution_client_split(labels_path, num_client, val_ratio = 0.2, overlap_ratio = 0.2, root_dir = ROOT_PATH, is_3d = False):
    """
    Split the dataset into clients with the same distribution of labels.
    """
    labels_df = pd.read_csv(labels_path)
    labels_df = preprocessing_labels(labels_df, root_dir = root_dir)    
    labels_df = prepare_data(labels_df)

    client_datasets = distributed_data_to_clients(labels_df, num_clients=num_client, overlap_ratio=overlap_ratio)

    client_datasets = create_train_test(client_datasets, val_ratio=val_ratio, root_dir=root_dir, is_3d = is_3d)

    return client_datasets


#  Model

## 3D DenseNet

In [9]:
import torch.nn as nn 
import torch 
import torch.nn.functional as F
from collections import OrderedDict
from typing import List, Tuple
from torchsummary import summary


class _DenseLayer(nn.Sequential):
    def __init__(self, num_input_features, growth_rate, bn_size, drop_rate):
        super().__init__()
        self.add_module('norm1', nn.BatchNorm3d(num_input_features))
        self.add_module('relu1', nn.ReLU(inplace=True))
        self.add_module(
            'conv1',
            nn.Conv3d(num_input_features,
                      bn_size * growth_rate,
                      kernel_size=1,
                      stride=1,
                      bias=False))
        self.add_module('norm2', nn.BatchNorm3d(bn_size * growth_rate))
        self.add_module('relu2', nn.ReLU(inplace=True))
        self.add_module(
            'conv2',
            nn.Conv3d(bn_size * growth_rate,
                      growth_rate,
                      kernel_size=3,
                      stride=1,
                      padding=1,
                      bias=False))
        self.drop_rate = drop_rate

    def forward(self, x):
        new_features = super().forward(x)
        if self.drop_rate > 0:
            new_features = F.dropout(new_features,
                                     p=self.drop_rate,
                                     training=self.training)
        return torch.cat([x, new_features], 1)

class _DenseBlock(nn.Sequential):
    def __init__(self, num_layers, num_input_features, bn_size, growth_rate, drop_rate):
        super().__init__()
        for i in range(num_layers):
            layer = _DenseLayer(num_input_features + i * growth_rate,
                                growth_rate, bn_size, drop_rate)
            self.add_module('denselayer{}'.format(i + 1), layer)

class _Transition(nn.Sequential):
    def __init__(self, num_input_features, num_output_features):
        super().__init__()
        self.add_module('norm', nn.BatchNorm3d(num_input_features))
        self.add_module('relu', nn.ReLU(inplace=True))
        self.add_module(
            'conv',
            nn.Conv3d(num_input_features,
                      num_output_features,
                      kernel_size=1,
                      stride=1,
                      bias=False))
        self.add_module('pool', nn.AvgPool3d(kernel_size=2, stride=2))

class DenseNet(nn.Module):
    def __init__(self,
                 n_input_channels=1,
                 conv1_t_size=7,
                 conv1_t_stride=1,
                 no_max_pool=False,
                 growth_rate=16,
                 block_config=(4, 8, 16, 12),
                 num_init_features=32,
                 bn_size=4,
                 drop_rate=0,
                 num_classes=1):
        super().__init__()

        # First convolution
        self.features = [('conv1',
                          nn.Conv3d(n_input_channels,
                                    num_init_features,
                                    kernel_size=(conv1_t_size, 7, 7),
                                    stride=(conv1_t_stride, 2, 2),
                                    padding=(conv1_t_size // 2, 3, 3),
                                    bias=False)),
                         ('norm1', nn.BatchNorm3d(num_init_features)),
                         ('relu1', nn.ReLU(inplace=True))]
        if not no_max_pool:
            self.features.append(
                ('pool1', nn.MaxPool3d(kernel_size=3, stride=2, padding=1)))
        self.features = nn.Sequential(OrderedDict(self.features))

        # Each denseblock
        num_features = num_init_features
        for i, num_layers in enumerate(block_config):
            block = _DenseBlock(num_layers=num_layers,
                                num_input_features=num_features,
                                bn_size=bn_size,
                                growth_rate=growth_rate,
                                drop_rate=drop_rate)
            self.features.add_module('denseblock{}'.format(i + 1), block)
            num_features = num_features + num_layers * growth_rate
            if i != len(block_config) - 1:
                trans = _Transition(num_input_features=num_features,
                                    num_output_features=num_features // 2)
                self.features.add_module('transition{}'.format(i + 1), trans)
                num_features = num_features // 2

        self.features.add_module('norm5', nn.BatchNorm3d(num_features))
        self.classifier = nn.Linear(num_features, num_classes)

        # Khởi tạo trọng số
        for m in self.modules():
            if isinstance(m, nn.Conv3d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm3d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        features = self.features(x)
        out = F.relu(features, inplace=True)
        out = F.adaptive_avg_pool3d(out, output_size=(1, 1, 1)).view(features.size(0), -1)
        logits = self.classifier(out)
        return logits
    

In [10]:
import lightning as pl
import torch.nn as nn 
from torchmetrics import Accuracy, F1Score, Precision, Recall, MeanMetric
import torch.optim as optim 
import torch 



class DenseNetModule(pl.LightningModule):
    def __init__(self, net, learning_rate=1e-3, weight_decay = 1e-2, batch_size = 32):
        super().__init__()
        self.model = net
        self.learning_rate = learning_rate
        self.weight_decay = weight_decay
        self.batch_size = batch_size
        # how confidence model is in it prediction
        # tức model có thể rất tự tin trong quyết định nhưng thực tế lại sai
        # BCE = y*log(y_pred) + (1 - y)*log(1 - y_pred)
        self.criterion = nn.BCEWithLogitsLoss()
        
        self.train_accuracy = Accuracy(task="binary", num_classes=1)
        self.val_accuracy = Accuracy(task="binary", num_classes=1)
        self.test_accuracy = Accuracy(task="binary", num_classes=1)


        self.val_precision = Precision(task="binary", num_classes=1)
        self.test_precision = Precision(task="binary", num_classes=1)

        self.val_recall = Recall(task="binary", num_classes=1)
        self.test_recall = Recall(task="binary", num_classes=1)

        self.val_f1 = F1Score(task="binary", num_classes=1)
        self.test_f1 = F1Score(task="binary", num_classes=1)

    

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        # x.shape  = (batch_size, in_channel, height, width, depth), y.shape = (batch_size)
        logits = self(x.unsqueeze(1))
        
        loss = self.criterion(logits, y.float().unsqueeze(1))
        
        acc = self.train_accuracy((torch.sigmoid(logits) > 0.5).float(), y.unsqueeze(1))

        
        self.log('train/loss', loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log('train/acc', acc, on_step=False, on_epoch=True, prog_bar=True)
        
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x.unsqueeze(1))
        
        loss = self.criterion(logits, y.float().unsqueeze(1))
        acc = self.val_accuracy((torch.sigmoid(logits) > 0.5).float(), y.unsqueeze(1))
        f1 = self.val_f1((torch.sigmoid(logits) > 0.5).float(), y.unsqueeze(1))
        precision = self.val_precision((torch.sigmoid(logits) > 0.5).float(), y.unsqueeze(1))
        recall = self.val_recall((torch.sigmoid(logits) > 0.5).float(), y.unsqueeze(1))
        
        self.log('val/loss', loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log('val/acc', acc, on_step=False, on_epoch=True, prog_bar=True)
        self.log('val/f1', f1, on_step=False, on_epoch=True, prog_bar=True)
        self.log('val/precision', precision, on_step=False, on_epoch=True, prog_bar=True)
        self.log('val/recall', recall, on_step=False, on_epoch=True, prog_bar=True)

        return loss



    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x.unsqueeze(1))
        
        loss = self.criterion(logits, y.float().unsqueeze(1))
        acc = self.test_accuracy((torch.sigmoid(logits) > 0.5).float(), y.unsqueeze(1))
        f1 = self.test_f1((torch.sigmoid(logits) > 0.5).float(), y.unsqueeze(1))
        precision = self.test_precision((torch.sigmoid(logits) > 0.5).float(), y.unsqueeze(1))
        recall = self.test_recall((torch.sigmoid(logits) > 0.5).float(), y.unsqueeze(1))

        



        self.log('test/loss', loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log('test/acc', acc, on_step=False, on_epoch=True, prog_bar=True)
        self.log('test/f1', f1, on_step=False, on_epoch=True, prog_bar=True)
        self.log('test/precision', precision, on_step=False, on_epoch=True, prog_bar=True)
        self.log('test/recall', recall, on_step=False, on_epoch=True, prog_bar=True)
        return loss




    def configure_optimizers(self):
        optimizer =  torch.optim.AdamW(self.parameters(), lr=self.learning_rate, weight_decay = self.weight_decay)
        scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma = 0.95)

        return {
           "optimizer": optimizer,
           "lr_scheduler": {
               "scheduler": scheduler,
               "monitor": "val_loss",
           },
        }


## 2D model 

In [11]:
class BrainMRINet(nn.Module):
    def __init__(self, num_classes=2, input_size=(130, 130)):
        super(BrainMRINet, self).__init__()

        self.features = nn.Sequential(
            # Block 1
            nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),  

            # Block 2
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),  

            # Block 3
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),  
            
            # Block 4 
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2), 
            
            # Block 5 
            nn.AdaptiveAvgPool2d((1, 1))  # Output: 128 x 1 x 1
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),  # Output: 128
            nn.Linear(128, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

In [12]:
class BrainMRILightningModule(pl.LightningModule): 
    
    def __init__(self, net: nn.Module,  learning_rate=1e-3, weight_decay = 1e-2, batch_size = 32 ):
        super().__init__()

        self.save_hyperparameters(ignore=['net'])

        self.model = net 
        self.learning_rate = learning_rate
        self.weight_decay = weight_decay
        self.batch_size = batch_size


        self.train_acc = Accuracy(task= "multiclass",  num_classes=net.classifier[-1].out_features)
        self.val_acc = Accuracy(task= "multiclass", num_classes=net.classifier[-1].out_features)
        
        self.test_acc = Accuracy(task= "multiclass",  num_classes=net.classifier[-1].out_features)
        
        # F1 score 
        self.val_f1 = F1Score(task="multiclass", num_classes=net.classifier[-1].out_features)
        self.test_f1 = F1Score(task="multiclass", num_classes=net.classifier[-1].out_features)
        # Precision
        self.val_precision = Precision(task="multiclass", num_classes=net.classifier[-1].out_features)   
        self.test_precision = Precision(task="multiclass", num_classes=net.classifier[-1].out_features)
        # Recall
        self.val_recall = Recall(task="multiclass", num_classes=net.classifier[-1].out_features)
        self.test_recall = Recall(task="multiclass", num_classes=net.classifier[-1].out_features)

        self.criterion = nn.CrossEntropyLoss()


    def forward(self, x):
        return self.model(x)

    def configure_optimizers(self):
        optimizer =  optim.Adam(self.parameters(), lr=self.learning_rate, weight_decay=self.weight_decay)
        return {
            "optimizer": optimizer,
            "gradient_clip_val": 1.0,  # Adjust value as needed
        }
    

    def training_step(self, batch, batch_idx):
        x, y = batch
        outputs = self(x)
        
        loss = nn.CrossEntropyLoss()(outputs, y)
        preds = torch.argmax(outputs, dim=1)
        
        # Update metrics
        acc = self.train_acc(preds, y)
        

        self.log("train/loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log("train/acc", acc, on_step=False, on_epoch=True, prog_bar=True)
        
        return loss
    

    def validation_step(self, batch, batch_idx):
        x, y = batch
        outputs = self(x)
        
        loss = self.criterion(outputs, y)
        preds = torch.argmax(outputs, dim=1)
        
        # Update metrics
       
        acc =  self.val_acc(preds, y)
        f1 =  self.val_f1(preds, y)
        precision =  self.val_precision(preds, y)
        recall =  self.val_recall(preds, y)
        
        # Log metrics
        self.log("val/loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log("val/acc", acc, on_step=False, on_epoch=True, prog_bar=True)
        self.log("val/f1", f1, on_step=False, on_epoch=True, prog_bar=True)
        self.log("val/precision", precision, on_step=False, on_epoch=True, prog_bar=True)
        self.log("val/recall", recall, on_step=False, on_epoch=True, prog_bar=True)

        return loss
    

    def test_step(self, batch, batch_idx):
        x, y = batch
        outputs = self(x)
        
        loss = self.criterion(outputs, y)
        preds = torch.argmax(outputs, dim=1)
        
        # Update metrics
        acc = self.test_acc(preds, y)
        f1 = self.test_f1(preds, y)
        precision = self.test_precision(preds, y)
        recall = self.test_recall(preds, y)
        
        # Log metrics
        self.log("test/loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log("test/acc", acc, on_step=False, on_epoch=True, prog_bar=True)
        self.log("test/f1", f1, on_step=False, on_epoch=True, prog_bar=True)
        self.log("test/precision", precision, on_step=False, on_epoch=True, prog_bar=True)
        self.log("test/recall", recall, on_step=False, on_epoch=True, prog_bar=True)

        return loss

# Client

In [13]:
import torch 
from collections import OrderedDict
from flwr.client import NumPyClient
from flwr.common import  Context
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.nn as nn 
import logging 
import lightning as pl
import warnings 
from lightning.pytorch.loggers.wandb import WandbLogger
from lightning.pytorch.callbacks import ModelCheckpoint



2025-06-04 17:06:20.375417: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749056780.571058      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749056780.623766      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [14]:
class FlowerLightningClient(NumPyClient):


    def __init__(self, model: pl.LightningModule, train_dataloader, val_dataloader, epochs, batch_size, device, client_id): 

        self.model = model
        self.train_dataloader = train_dataloader
        self.val_dataloader = val_dataloader
        self.epochs = epochs
        self.device = device 
        self.client_id = client_id
        self.batch_size = batch_size
    
    def get_parameters(self, config):

        return [val.cpu().numpy() for _, val in self.model.state_dict().items()]

    def set_parameters(self, parameters):
        if not parameters:
            return

        params_dict = zip(self.model.state_dict().keys(), parameters)
        state_dict = OrderedDict()
        for k, v in params_dict:
            state_dict[k] = torch.tensor(v)
            

        if state_dict:
            self.model.load_state_dict(state_dict, strict=False)

        
    def fit(self, parameters, config):

        self.set_parameters(parameters)
        
        checkpoint_callback = ModelCheckpoint(
            dirpath=f"./checkpoints/client_{self.client_id}",
            filename=f"round_{config.get('round_num', 0)}" + "-{epoch:02d}",
            save_top_k=1,
            monitor="val/loss",
            mode="min"
        )

        trainer = pl.Trainer(
            max_epochs=self.epochs,
            accelerator="auto",
            devices=1,
            callbacks=[checkpoint_callback],
            enable_progress_bar=False, 
            log_every_n_steps=1
        )

        trainer.fit(self.model, train_dataloaders=self.train_dataloader, val_dataloaders=self.val_dataloader)

        callback_metrics = trainer.callback_metrics

        train_loss = callback_metrics.get("train/loss", 0)
        train_accuracy = callback_metrics.get("train/acc", 0)
        val_loss = callback_metrics.get("val/loss", 0)
        val_accuracy = callback_metrics.get("val/acc", 0)
        val_precision = callback_metrics.get("val/precision", 0)
        val_recall = callback_metrics.get("val/recall", 0)
        val_f1 = callback_metrics.get("val/f1", 0)



        metrics = {
            "train_loss": train_loss.item() if isinstance(train_loss, torch.Tensor) else float(train_loss),
            "train_accuracy": train_accuracy.item() if isinstance(train_accuracy, torch.Tensor) else float(train_accuracy),
            "val_loss": val_loss.item() if isinstance(val_loss, torch.Tensor) else float(val_loss),
            "val_accuracy": val_accuracy.item() if isinstance(val_accuracy, torch.Tensor) else float(val_accuracy),
            "val_precision": val_precision.item() if isinstance(val_precision, torch.Tensor) else float(val_precision),
            "val_recall": val_recall.item() if isinstance(val_recall, torch.Tensor) else float(val_recall),
            "val_f1": val_f1.item() if isinstance(val_f1, torch.Tensor) else float(val_f1)
        }



        return self.get_parameters(config={}), len(self.train_dataloader.dataset), metrics


    def evaluate(self, parameters, config):

        self.set_parameters(parameters)

       
        trainer = pl.Trainer(
            accelerator="auto",
            devices=1 ,
            enable_progress_bar=False
        )

        results = trainer.test(self.model, dataloaders=self.val_dataloader)
        
        callback_metrics = trainer.callback_metrics

        test_loss = callback_metrics.get("test/loss", 0)
        test_accuracy = callback_metrics.get("test/acc", 0)
        test_f1 = callback_metrics.get("test/f1", 0)
        test_precision = callback_metrics.get("test/precision", 0)
        test_recall = callback_metrics.get("test/recall", 0)
        
        # Additional metrics
        metrics = {
            "test_loss": test_loss.item() if isinstance(test_loss, torch.Tensor) else float(test_loss),
            "test_accuracy": test_accuracy.item() if isinstance(test_accuracy, torch.Tensor) else float(test_accuracy),
            "test_f1": test_f1.item() if isinstance(test_f1, torch.Tensor) else float(test_f1),
            "test_precision": test_precision.item() if isinstance(test_precision, torch.Tensor) else float(test_precision),
            "test_recall": test_recall.item() if isinstance(test_recall, torch.Tensor) else float(test_recall)
        }

       
        return float(test_loss), len(self.val_dataloader.dataset), metrics


In [15]:
def create_lightning_client_fn(device, epochs, client_datasets, batch_size, num_workers, pl_model):

    def client_fn(context: Context):
        
        
        client_id = context.node_config['partition-id']
        train_dataset, val_dataset = client_datasets[client_id]

        train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True, num_workers = num_workers)
        val_dataloader = DataLoader(val_dataset, batch_size = batch_size, shuffle = False, num_workers = num_workers)
        return FlowerLightningClient(pl_model, train_dataloader, val_dataloader, epochs, batch_size, device, client_id).to_client()

    return client_fn

In [16]:
import random
import numpy as np
from typing import Dict, List, Optional, Tuple, Union
import flwr as fl
from flwr.common import (
    EvaluateRes,
    FitIns,
    FitRes,
    Parameters,
    EvaluateIns,
)
from flwr.server.client_proxy import ClientProxy
from flwr.server.client_manager import ClientManager
from flwr.server.strategy import FedAvg
from flwr.common.parameter import parameters_to_ndarrays
import numpy as np 
import torch
from collections import OrderedDict
import wandb


In [17]:
class DropoutFedAvg(FedAvg):
    """FedAvg strategy with client dropout simulation and metrics tracking."""

    def __init__( self, net, dropout_rate_training: float = 0.3, dropout_rate_eval: float = 0.3, fixed_clients: Optional[List[int]] = None, dropout_pattern_train: str = "random", dropout_pattern_eval: str = "random", **kwargs):
    
        if "fit_metrics_aggregation_fn" not in kwargs:
            kwargs["fit_metrics_aggregation_fn"] = self.weighted_average
        if "evaluate_metrics_aggregation_fn" not in kwargs:
            kwargs["evaluate_metrics_aggregation_fn"] = self.weighted_average

        super().__init__(**kwargs)
        self.dropout_rate_training = dropout_rate_training
        self.dropout_rate_eval = dropout_rate_eval
        self.fixed_clients = fixed_clients or []
        self.dropout_pattern_train = dropout_pattern_train
        self.dropout_pattern_eval = dropout_pattern_eval
        self.current_round = 0
        self.dropped_clients_history_training: Dict[int, List[int]] = {}
        self.dropped_clients_history_evaluation: Dict[int, List[int]] = {}

        # For tracking metrics
        self.fit_metrics_history: List[Dict[str, float]] = []
        self.eval_metrics_history: List[Dict[str, float]] = []

        self.net = net
    


    def weighted_average(self, metrics: List[Tuple[int, Dict]]) -> Dict:
        """Aggregate metrics using weighted average based on number of samples."""
        if not metrics:
            return {}

        total_examples = sum([num_examples for num_examples, _ in metrics])
        weighted_metrics = {}

        for metric_key in metrics[0][1].keys():
            weighted_sum = sum(
                metric_dict[metric_key] * num_examples
                for num_examples, metric_dict in metrics
                if metric_key in metric_dict
            )
            weighted_metrics[metric_key] = weighted_sum / total_examples if total_examples > 0 else 0

        return weighted_metrics


    def configure_fit( self, server_round: int, parameters: Parameters, client_manager: ClientManager) -> List[Tuple[ClientProxy, FitIns]]:
        """Configure the next round of training with client dropout."""
        self.current_round = server_round

    
        client_fit_instructions = super().configure_fit(
            server_round, parameters, client_manager
        )

        if not client_fit_instructions:
            return []


        available_clients = self._apply_dropout(client_fit_instructions, dropout_rate=self.dropout_rate_training, dropout_pattern=self.dropout_pattern_train)

        # Save dropout history for this round
        client_ids = [int(client.cid) for client, _ in client_fit_instructions]
        available_client_ids = [int(client.cid) for client, _ in available_clients]
        dropped_clients = [cid for cid in client_ids if cid not in available_client_ids]
        self.dropped_clients_history_training[server_round] = dropped_clients

        wandb.log({
            "train_dropout_history"  : len(dropped_clients)
        })

        print(f"Round {server_round}: {len(dropped_clients)} clients dropped out of {len(client_ids)} during training")
        print(f"Dropped client IDs: {dropped_clients}")

        return available_clients
    



    def configure_evaluate( self, server_round: int, parameters: Parameters, client_manager: ClientManager) -> List[Tuple[ClientProxy, EvaluateIns]]:
        self.current_round = server_round

        client_evaluate_instructions = super().configure_evaluate(
            server_round, parameters, client_manager
        )

        if not client_evaluate_instructions: return []

        available_clients = self._apply_dropout(client_evaluate_instructions, dropout_rate=self.dropout_rate_eval, dropout_pattern=self.dropout_pattern_eval)

        client_ids = [int(client.cid) for client, _ in client_evaluate_instructions]
        available_client_ids = [int(client.cid) for client, _ in available_clients]
        dropped_clients = [cid for cid in client_ids if cid not in available_client_ids]

        self.dropped_clients_history_evaluation[server_round] = dropped_clients
        wandb.log({
            "eval_dropout_history" : len(dropped_clients) 
        })

        print(f"Round {server_round}: {len(dropped_clients)} clients dropped out of {len(client_ids)} during evaluation")
        print(f"Dropped client IDs: {dropped_clients}")

        return available_clients




    def _apply_dropout(self, client_instructions: List[Tuple[ClientProxy, Union[FitIns, EvaluateIns ]]], dropout_pattern: str, dropout_rate: 0.3) -> List[Tuple[ClientProxy, FitIns]]:
        """Apply dropout to clients based on the specified pattern."""
        if len(client_instructions) == 0:
            return []

        # Get all client IDs
        all_clients = [(client, ins) for client, ins in client_instructions]
        all_client_ids = [int(client.cid) for client, _ in all_clients]

        # Determine which clients will drop out
        dropout_mask = [False] * len(all_clients)

        if dropout_pattern == "random":
           
            for i, cid in enumerate(all_client_ids):
                
                if cid in self.fixed_clients:
                    continue
            
                if random.random() < dropout_rate:
                    dropout_mask[i] = True

        elif dropout_pattern == "alternate":
         
            if self.current_round % 2 == 1:  
                for i, cid in enumerate(all_client_ids):
                    if cid not in self.fixed_clients:
                        dropout_mask[i] = True

        elif dropout_pattern == "fixed":
      
            n_dropout = int(len(all_clients) * dropout_rate)
            for i in range(n_dropout):
                if all_client_ids[i] not in self.fixed_clients:
                    dropout_mask[i] = True

        
        available_clients = [
            (client, ins) for i, (client, ins) in enumerate(all_clients)
            if not dropout_mask[i]
        ]

        return available_clients

    def aggregate_fit(self, server_round: int, results: List[Tuple[ClientProxy, FitRes]], failures: List[Union[Tuple[ClientProxy, FitRes], BaseException]]):
    
        aggregated = super().aggregate_fit(server_round, results, failures)

        if aggregated and aggregated[0] is not None:
            aggregated_ndarrays: list[np.ndarray] = fl.common.parameters_to_ndarrays(
                aggregated[0]
            )

            params_dict = zip(self.net.state_dict().keys(), aggregated_ndarrays)
            state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})
            self.net.load_state_dict(state_dict, strict=True)

            torch.save(self.net.state_dict(), f"model_round_{server_round}.pth")

        if results:
            metrics = [(res.num_examples, res.metrics) for _, res in results]
            aggregated_metrics = self.weighted_average(metrics)
            self.fit_metrics_history.append(aggregated_metrics)

            wandb.log({
                "train_server_round": server_round, 
                "train_accuracy": aggregated_metrics.get("train_accuracy", 0.0), 
                "train_loss" : aggregated_metrics.get("train_loss", 0.0)
            })


            print(f"Round {server_round} training metrics: {aggregated_metrics}")

        return aggregated

    def aggregate_evaluate( self, server_round: int, results: List[Tuple[ClientProxy, EvaluateRes]],  failures: List[Union[Tuple[ClientProxy, EvaluateRes], BaseException]]):
        
        aggregated = super().aggregate_evaluate(server_round, results, failures)

        if results:
            metrics = [(res.num_examples, res.metrics) for _, res in results]
            aggregated_metrics = self.weighted_average(metrics)
            self.eval_metrics_history.append(aggregated_metrics)

            print(f"Round {server_round} evaluation metrics: {aggregated_metrics}")

            wandb.log({
                "server_round_eval" : server_round,
                "test_accuracy": aggregated_metrics.get("test_accuracy", 0.0), 
                "test_loss" : aggregated_metrics.get("test_loss", 0.0), 
                 "test_f1": aggregated_metrics.get("test_f1", 0.0), 
                "test_precision" : aggregated_metrics.get("test_precision", 0.0), 
                 "test_recall": aggregated_metrics.get("test_recall", 0.0), 
            })

        return aggregated

    def get_dropout_history(self) -> Dict[int, List[int]]:
        return self.dropped_clients_history_training, self.dropped_clients_history_evaluation

    def get_metrics_history(self) -> Tuple[List[Dict[str, float]], List[Dict[str, float]]]: 
        return self.fit_metrics_history, self.eval_metrics_history

In [18]:
from flwr.client import ClientApp
from flwr.server import ServerApp
from flwr.simulation import run_simulation
import pandas as pd
import matplotlib.pyplot as plt
from typing import Dict, List, Optional
import os
from flwr.common import Context
import torch 
import lightning as pl
from typing import Union

In [19]:
def run_dropout_experiment(
    client_fn_creator,
    pl_model : Union[pl.LightningModule, torch.nn.Module], 
    num_clients: int,
    num_rounds: int = 5,
    dropout_rate_training: float = 0.3,
    dropout_rate_eval: float = 0.3,
    dropout_pattern_train: str = "random",
    dropout_pattern_eval: str = "random",
    fixed_clients: Optional[List[int]] = None,
    experiment_name: str = "dropout_experiment",
    save_dir: str = "model_weights",
    num_gpus : int = 0, 
    resource_config : Optional[Dict[str, float]] = None,

):
    
      # Configure client app
    print(f"\nStarting experiment: {experiment_name}")
    print(f"Dropout rate training: {dropout_rate_training}, Pattern: {dropout_pattern_train}")
    print(f"Dropout rate evaluation: {dropout_rate_eval}, Pattern: {dropout_pattern_eval   }")
    print(f"Number of GPUs: {num_gpus}")
    print(f"Number of clients: {num_clients}")
    print(f"Number of rounds: {num_rounds}")
    print(f"Fixed clients: {fixed_clients or []}")

    # Create strategy with dropout
    strategy = DropoutFedAvg(
        net=pl_model.model if isinstance(pl_model, pl.LightningModule) else pl_model,
        dropout_rate_training=dropout_rate_training,
        dropout_rate_eval=dropout_rate_eval,
        dropout_pattern_train=dropout_pattern_train,
        dropout_pattern_eval=dropout_pattern_eval,
        fixed_clients=fixed_clients or [],
        fraction_fit=1.0,
        fraction_evaluate=1.0,
        min_fit_clients=1,
        min_evaluate_clients=1,
        min_available_clients=1,

    )

    # Configure server with strategy
    def server_fn(server_context: Context):
        from flwr.server import ServerAppComponents, ServerConfig
        config = ServerConfig(num_rounds=num_rounds)
        return ServerAppComponents(strategy=strategy, config=config)


    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    epochs = resource_config.get("epochs", 1) if resource_config else 1
    client_datasets = resource_config.get("client_datasets", {}) if resource_config else {}

    
    batch_size = resource_config.get("batch_size", 32) if resource_config else 32
    learning_rate = resource_config.get("learning_rate", 0.001) if resource_config else 0.001
    num_workers = resource_config.get("num_workers", 1) if resource_config else 1
    client_fn = client_fn_creator(device=device, epochs=epochs, client_datasets=client_datasets
                                , batch_size=batch_size, pl_model=pl_model, num_workers=num_workers)
    
    # Create client and server apps
    client_app = ClientApp(client_fn=client_fn)
    server_app = ServerApp(server_fn=server_fn)

    # Configure backend
    backend_config = {
        "client_resources": {
            "num_cpus": 1,
            "num_gpus": num_gpus,
        }
    }
    history = strategy.get_dropout_history()
    # Run simulation
    try:
        run_simulation(
            client_app=client_app,
            server_app=server_app,
            num_supernodes=num_clients,
            backend_config=backend_config,
        )

        # Get metrics directly from strategy
        fit_metrics, eval_metrics = strategy.get_metrics_history()

        # Format metrics for plotting
        rounds = list(range(1, len(eval_metrics) + 1))

        train_accuracy_values = [metrics.get("train_accuracy", 0.0) for metrics in fit_metrics]
        train_loss_values = [metrics.get("train_loss", 0.0) for metrics in fit_metrics]
        

        test_accuracy_values = [metrics.get("test_accuracy", 0.0) for metrics in eval_metrics]
        test_loss_values = [metrics.get("test_loss", 0.0) for metrics in eval_metrics]
        test_f1_values = [metrics.get("test_f1", 0.0) for metrics in eval_metrics]
        test_precision_values = [metrics.get("test_precision", 0.0) for metrics in eval_metrics]
        test_recall_values = [metrics.get("test_recall", 0.0) for metrics in eval_metrics]    


        # cleanup_wandb_loggers()

        results = {
            "rounds": rounds,
            "train_accuracy": train_accuracy_values,
            "train_loss": train_loss_values,

            "test_accuracy": test_accuracy_values,
            "test_loss": test_loss_values,
            "test_f1": test_f1_values,
            "test_precision": test_precision_values,
            "test_recall": test_recall_values
        }

        return results, history
    
    except Exception as e:
        print(f"Error in dropout experiment: {e}")
        import traceback
        traceback.print_exc()
        return {"error": str(e)}

# Main

In [20]:
from typing import List, Dict, Tuple, Optional, Union
import torch.nn as nn 
import hydra 
from omegaconf import DictConfig, OmegaConf
import logging 
import wandb 
import os 
import warnings
import lightning as pl 

## Config 

In [21]:
config = {
    "base_path": "/kaggle",
    "device": "cuda",  # from train.device
    "run_name": "dropout_30pct_fixed_train_fixed_eval_0.3_fixed_0.3",  # resolved using experiment values
    "seed": 42,
    "num_clients": 10,
    "num_rounds": 10,
    "gpus": 1,
    
    "train": {
        "batch_size": 4,
        "learning_rate": 0.001,
        "epochs": 1,
        "device": "cuda",
        "num_workers": 2,
        "weight_decay": 0.0001,
        "scheduler": {
            "use": False,
            "type": "cosine",
            "warmup_epochs": 5,
            "min_lr": 0.0001,
        }
    },

    "experiment": {
        "pattern_train": "random",
        "pattern_eval": "random",
        "dropout_rate_training": 0.3,
        "dropout_rate_eval": 0.3,
        "fixed_clients": [0, 1, 2],
        "name": "dropout_30pct_random_train_random_eval",
    },

    "data": {
        "root_path": "/kaggle/input/mri-dataset/datasetzip/not_skull_stripped",
        "label_path": "/kaggle/input/mri-label/label.csv",
        "val_ratio": 0.2,
        "overlap_ratio": 0.2,
        "distribution": "same",
    },
    "is_3d": False
}


In [22]:
def run_experiment_with_lightning(cfg_dict: dict) -> None:
    cfg: DictConfig = OmegaConf.create(cfg_dict)  # convert to DictConfig to retain dot-access
    logger =  logger = logging.getLogger(__name__)
    logger.info(f"Running experiment with config: {cfg.experiment.name}")
    logger.info(f"Config: {OmegaConf.to_yaml(cfg)}")

    wandb.login(
        key=WANDB_APIKEY 
    )

    wandb.init(
        project="federated-mri-server",
        name=f"{cfg.experiment.name}",
        config=OmegaConf.to_container(cfg, resolve=True),
        group="server"
    )

    device = cfg.device
    epochs = cfg.train.epochs

    logger.info("Loading model")

    net : nn.Module() = BrainMRINet()
    pl_model : pl.LightningModule = BrainMRILightningModule(net = net )


    
    # net: nn.Module = DenseNet()
    # pl_model: pl.LightningModule = DenseNetModule(net = net)

    logger.info("Loading dataset")
    is_3d = True if isinstance(pl_model, DenseNetModule) else False
    print(f"Is 3D: {is_3d}")

   
    logger.info(f"Splitting dataset into {cfg.num_clients} clients")

    if cfg.data.distribution == "iid":
        client_datasets = iid_client_split(
            full_dataset,
            num_client=cfg.num_clients,
            val_ratio=cfg.data.val_ratio
        )
    elif cfg.data.distribution == "same":
        client_datasets = same_distribution_client_split(
            cfg.data.label_path,
            num_client=cfg.num_clients,
            val_ratio=cfg.data.val_ratio,
            overlap_ratio=cfg.data.overlap_ratio,
            root_dir=cfg.data.root_path,
            is_3d=is_3d
        )
    else:
        raise ValueError(f"Unknown distribution type: {cfg.data.distribution}")

    logger.info(f"Client datasets created successfully with {len(client_datasets)} clients")

    resources = {
        "client_datasets": client_datasets,
        "device": device,
        "epochs": epochs,
        "batch_size": cfg.train.batch_size,
        "learning_rate": cfg.train.learning_rate,
        "num_workers": cfg.train.num_workers
    }

    logger.info("Running experiments with PyTorch Lightning")

    results, history = run_dropout_experiment(
        client_fn_creator=create_lightning_client_fn,
        pl_model=pl_model,
        num_clients=cfg.num_clients,
        num_rounds=cfg.num_rounds,
        dropout_rate_training=cfg.experiment.dropout_rate_training,
        dropout_rate_eval=cfg.experiment.dropout_rate_eval,
        dropout_pattern_train=cfg.experiment.pattern_train,
        dropout_pattern_eval=cfg.experiment.pattern_eval,
        experiment_name=cfg.experiment.name,
        num_gpus=cfg.gpus,
        resource_config=resources
    )

    logger.info("Run successfully + wandb tracking")
    print(f"Result is {results}")
    wandb.finish()

    logger.info("Experiments completed successfully")
    logger.info(f"Client Dropout History: {history}")

    return results, history

In [23]:
run_experiment_with_lightning(config)

/usr/local/lib/python3.11/dist-packages/notebook/utils.py:280: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  return LooseVersion(v) >= LooseVersion(check)
/usr/local/lib/python3.11/dist-packages/google/colab/_import_hooks/_bokeh.py:16: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import imp  # pylint: disable=deprecated-module
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: theseventeengv (trungviet17)

Is 3D: False


/tmp/ipykernel_35/607595986.py:28: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  samples = samples.apply(lambda x: x.sample(
/tmp/ipykernel_35/607595986.py:28: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  samples = samples.apply(lambda x: x.sample(
/tmp/ipykernel_35/607595986.py:28: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future ver


Starting experiment: dropout_30pct_random_train_random_eval
Dropout rate training: 0.3, Pattern: random
Dropout rate evaluation: 0.3, Pattern: random
Number of GPUs: 1
Number of clients: 10
Number of rounds: 10
Fixed clients: []


(pid=452) 2025-06-04 17:17:51.089640: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=452) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
(pid=452) E0000 00:00:1749057471.113045     452 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=452) E0000 00:00:1749057471.120057     452 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
INFO :      Received initial parameters from one random client
INFO :      Starting evaluation of initial global parameters
INFO :      Evaluation returned no results (`None`)
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 8 clients (out of 10)


...

(ClientAppActor pid=452) ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
(ClientAppActor pid=452) ┃        Test metric        ┃       DataLoader 0        ┃
(ClientAppActor pid=452) ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
(ClientAppActor pid=452) │         test/acc          │    0.3762376308441162     │
(ClientAppActor pid=452) │          test/f1          │    0.3762376308441162     │
(ClientAppActor pid=452) │         test/loss         │    0.7969539165496826     │
(ClientAppActor pid=452) │      test/precision       │    0.3762376308441162     │
(ClientAppActor pid=452) │        test/recall        │    0.3762376308441162     │
(ClientAppActor pid=452) └───────────────────────────┴───────────────────────────┘
Round 10 evaluation metrics: {'test_loss': 0.6611779988414109, 'test_accuracy': 0.5396825481114793, 'test_f1': 0.5396825481114793, 'test_precision': 0.5396825481114793, 'test_recall': 0.5396825481114793}


INFO :      	               (2, 0.6894544849096629),
INFO :      	               (3, 0.6939519975985233),
INFO :      	               (4, 0.6923129401117001),
INFO :      	               (5, 0.6990406974576987),
INFO :      	               (6, 0.6796625331311243),
INFO :      	               (7, 0.680844415552341),
INFO :      	               (8, 0.6639552458039363),
INFO :      	               (9, 0.6138671744029501),
INFO :      	               (10, 0.6611779988414109)],
INFO :      	 'test_precision': [(1, 0.4452644560719768),
INFO :      	                    (2, 0.5645756509617832),
INFO :      	                    (3, 0.4501607720076335),
INFO :      	                    (4, 0.4675052355295457),
INFO :      	                    (5, 0.49399038389898264),
INFO :      	                    (6, 0.49462365730261715),
INFO :      	                    (7, 0.48557692447390693),
INFO :      	                    (8, 0.5239852479536449),
INFO :      	                    (9, 0.6918392158890467

Result is {'rounds': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'train_accuracy': [0.5596696986565263, 0.5915622177819381, 0.5819169085919575, 0.6186440663714385, 0.6312154784222335, 0.6672240726425496, 0.6912181250068576, 0.6888652169202704, 0.7345565755250621, 0.7340499221575905], 'train_loss': [0.6915231507606944, 0.6636595545613923, 0.669087836830204, 0.6536725372184276, 0.6523050912218529, 0.6304668008433936, 0.5976119376050627, 0.5868222142510676, 0.5479022490686597, 0.5490410326464662], 'test_accuracy': [0.4452644560719768, 0.5645756509617832, 0.4501607720076335, 0.4675052355295457, 0.49399038389898264, 0.49462365730261715, 0.48557692447390693, 0.5239852479536449, 0.6918392158890467, 0.5396825481114793], 'test_loss': [0.6918848821157899, 0.6894544849096629, 0.6939519975985233, 0.6923129401117001, 0.6990406974576987, 0.6796625331311243, 0.680844415552341, 0.6639552458039363, 0.6138671744029501, 0.6611779988414109], 'test_f1': [0.4452644560719768, 0.5645756509617832, 0.4501607720076335, 0.4

eval_dropout_history,▃▃▁█▃▃▃▃▃▅
server_round_eval,▁▂▃▃▄▅▆▆▇█
test_accuracy,▁▄▁▂▂▂▂▃█▄
test_f1,▁▄▁▂▂▂▂▃█▄
test_loss,▇▇█▇█▆▇▅▁▅
test_precision,▁▄▁▂▂▂▂▃█▄
test_recall,▁▄▁▂▂▂▂▃█▄
train_accuracy,▁▂▂▃▄▅▆▆██
train_dropout_history,▃▅▁▅▆█▁▃▅▆
train_loss,█▇▇▆▆▅▃▃▁▁
train_server_round,▁▂▃▃▄▅▆▆▇█


({'rounds': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
  'train_accuracy': [0.5596696986565263,
   0.5915622177819381,
   0.5819169085919575,
   0.6186440663714385,
   0.6312154784222335,
   0.6672240726425496,
   0.6912181250068576,
   0.6888652169202704,
   0.7345565755250621,
   0.7340499221575905],
  'train_loss': [0.6915231507606944,
   0.6636595545613923,
   0.669087836830204,
   0.6536725372184276,
   0.6523050912218529,
   0.6304668008433936,
   0.5976119376050627,
   0.5868222142510676,
   0.5479022490686597,
   0.5490410326464662],
  'test_accuracy': [0.4452644560719768,
   0.5645756509617832,
   0.4501607720076335,
   0.4675052355295457,
   0.49399038389898264,
   0.49462365730261715,
   0.48557692447390693,
   0.5239852479536449,
   0.6918392158890467,
   0.5396825481114793],
  'test_loss': [0.6918848821157899,
   0.6894544849096629,
   0.6939519975985233,
   0.6923129401117001,
   0.6990406974576987,
   0.6796625331311243,
   0.680844415552341,
   0.6639552458039363,
   0.6138671744